In [ ]:
%pip install plotly
%pip install --upgrade nbformat

## Building an AI Travel Assistant For Global Explorers

Watch Dev Day Demo 33:33
https://www.youtube.com/watch?v=U9mJuUkhUzk

Get MapBox Access Token:
https://www.mapbox.com/

In [292]:
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv
import os

_: bool = load_dotenv(find_dotenv())  # read local .env file

client: OpenAI = OpenAI()

MAPBOX_ACCESS_TOKEN = os.environ.get("MAPBOX_TOKEN")

### Seed Configuration For Assistant

In [293]:
from typing import Union

# Because dictionaries are mutable, any changes you make to a dictionary are done in place.
# Here we are them to store the state of map. It allows to retrieve the state and easily update it using function calling.

# When you use the global keyword in a function, it allows you to access and modify variables that are defined in the global scope from within the function. 

map_state: dict[str, Union[float, str]] = {
            "latitude": 39.949610,
            "longitude": -75.150282,
            "zoom": 16,}

markers_state: dict[str, list[Union[float, str]]] = {
    "latitudes": [],
    "longitudes": [],
    "labels": [],
}

# Define Functions For Our Assistant - Use Try Catch Blocks For Error Handling

# Function 1 - Control Map Location
def update_map(longitude: float, latitude: float, zoom: int) -> str:
    """Update map to center on a particular location."""

    global map_state  # Refer to the global map_state

    try: 
        if not (-90 <= latitude <= 90) or not (-180 <= longitude <= 180) or not (isinstance(zoom, int) and zoom > 0):
            raise ValueError("""Latitude must be between -90 and 90,
                             Longitude must be between -180 and 180,
                             and Zoom must be a positive integer.""")

        # Update the global map_state
        map_state['latitude'] = latitude
        map_state['longitude'] = longitude
        map_state['zoom'] = zoom

        return "Map Updated."
    except (ValueError, TypeError) as e:
        raise ValueError(f"Error in update_map function: {e}")

# Function 2 - Add markers to map
def add_markers(latitudes: list, longitudes: list, labels: list) -> str:
    """OpenAI tool to update markers in-app"""

    global markers_state  # Refer to the global map_state

    try: 
         # Validate inputs
        if len(latitudes) != len(longitudes) or len(latitudes) != len(labels):
            raise ValueError("Number of latitudes, longitudes, and labels must be the same.")

        markers_state["latitudes"] = latitudes
        markers_state["longitudes"] = longitudes
        markers_state["labels"] = labels
        
        return "Markers added successfully."
    
    except (ValueError, TypeError) as e:
        raise ValueError(f"Error in add_markers function: {e}")


available_functions = {
    "update_map": update_map,
    "add_markers": add_markers,
}

SEED_INSTRUCTION: str = """You are a reliable travel assistant, here to assist Global Explorers in planning and discovering their next travel 
                            destination. You specialize in marking locations on the map, providing personalized travel suggestions, and helping 
                            you reach any destination you have in mind. Wherever possible mark locations on map while making the travelers travel 
                            plans memorable. In map marker label share the destination names. For any uploaded pdf use "retrieval" to analyze them 
                            from an AI Travel Agent Prespective and Share the Data present in PDF in an organized format """


travel_agent_tools = [
    {"type": "retrieval"},
    {"type": "code_interpreter"},
    {
        "type": "function",
        "function": {
            "name": "update_map",
            "description": "Update map to center on a particular location",
            "parameters": {
                "type": "object",
                "properties": {
                    "longitude": {
                        "type": "number",
                        "description": "Longitude of the location to center the map on"
                    },
                    "latitude": {
                        "type": "number",
                        "description": "Latitude of the location to center the map on"
                    },
                    "zoom": {
                        "type": "integer",
                        "description": "Zoom level of the map"
                    }
                },
                "required": ["longitude", "latitude", "zoom"]
            }
        }
    },
    {
        "type": "function",
        "function":  {
            "name": "add_markers",
            "description": "Add list of markers to the map",
            "parameters": {
                "type": "object",
                "properties": {
                    "longitudes": {
                        "type": "array",
                        "items": {
                            "type": "number"
                        },
                        "description": "List of longitude of the location to each marker"
                    },
                    "latitudes": {
                        "type": "array",
                        "items": {
                            "type": "number"
                        },
                        "description": "List of latitude of the location to each marker"
                    },
                    "labels": {
                        "type": "array",
                        "items": {
                            "type": "string"
                        },
                        "description": "List of text to display on the location of each marker"
                    }
                },
                "required": ["longitudes", "latitudes", "labels"]
            }
        }
    },

]

#### Function To Plot and Update the Map

In [294]:
import plotly.graph_objects as go

# Function to create and display the map
def create_map(map_state: dict[str, Union[float, str]], markers_state: dict[str, list[Union[float, str]]]) -> None:
    
    figure = go.Figure(go.Scattermapbox(mode="markers"))
    
    figure.add_trace(
        go.Scattermapbox(
            mode="markers",
            marker=dict(color='red', size=14),
            lat=markers_state["latitudes"],
            lon=markers_state["longitudes"],
            text=markers_state["labels"]
        )
    )
    
    figure.update_layout(
        mapbox=dict(
            # style="open-street-map",
            accesstoken=MAPBOX_ACCESS_TOKEN, # If you don't have MAPBOX token, replace with style="open-street-map".
            center=go.layout.mapbox.Center(
                lat=map_state["latitude"],
                lon=map_state["longitude"]
            ),
            zoom=map_state["zoom"]
        ),
        margin=dict(l=0, r=0, t=0, b=0)
    )
    figure.show()

### Create an Assistant

In [295]:
from openai.types.beta.assistant import Assistant

travel_agent: Assistant = client.beta.assistants.create(
    model="gpt-4-1106-preview",
    name="AI Travel Agent",
    instructions=SEED_INSTRUCTION,
    tools=travel_agent_tools
)

# Create an Assistant Once and Store it's ID in the env variables. Next Here retrive the assistant and use it. You can modify it. 

### Create a Thread

In [296]:
from openai.types.beta.thread import Thread

thread: Thread = client.beta.threads.create()

### Manage File Operations For Retrival Efficiently

In [297]:
from openai.types.file_deleted import FileDeleted
from typing import Literal
from openai.types.file_list_params import FileListParams
from openai.types.file_object import FileObject

# For the retrival we will upload files and in message we will pass the file id.
# Remember to destroy the file afterwards

# For Cost Optimization Create a Thread with Attached File Once and then Use it for All Operations/Questions Related to That File

class TravelFiles():
    def __init__(self):
        self.client = OpenAI()

    def list_files(self, purpose: str = 'assistants') -> FileListParams:
        """Retrieve a list of files with the specified purpose."""
        files = client.files.list(purpose=purpose)
        file_list = files.model_dump()
        return file_list['data'] if 'data' in file_list else []

    def upload_file(self, file_path: str, purpose: Literal['fine-tune', 'assistants'] = 'assistants') -> str:
        """Create or find a file in OpenAI. 
        https://platform.openai.com/docs/api-reference/files/list
        Returns File ID. """

        with open(file_path, "rb") as file:
            file_obj: FileObject = self.client.files.create(file=file, purpose=purpose)
            self.file_id: str = file_obj.id
            return file_obj.id
            
    def deleteFile(self, file_id: str) -> dict[str, FileDeleted | str]:
        """Delete an Uploaded File
        args: Pass file Id
        returns feleted file id, object and deleted (bool)"""

        response: dict[str, FileDeleted | str] = {}
        try:
            response['data']: FileDeleted = self.client.files.delete(file_id)
            response['status']: str = 'success'
            print("Deleted File", response['data'])

        except Exception as e:
            # Handle other potential exceptions
            response['status'] = 'error'
            response['error'] = str(e)

        return response

### Class to Manage all Assistant Operations Efficiently

In [298]:
from openai.types.beta.threads import Run
from openai.types.beta.threads import ThreadMessage
import time
import json

class TravelAIChat():
    def __init__(self, assistant: Assistant, thread: Thread):
        self.client = OpenAI()
        self.assistant: Assistant | None = assistant
        self.thread: Thread | None = thread

    def modifyAssistant(self, new_instructions: str, tools: list, file_obj: list[str]) -> Assistant:
        """Update an existing assistant."""
        print("Updating edisting assistant...")
        self.assistant = self.client.beta.assistants.update(
            instructions=new_instructions,
            tools=tools,
            model=self.model,
            file_ids=file_obj
        )
        return self.assistant

    def add_message_to_thread(self, role: Literal['user'], content: str, file_obj_ids: list[str] = '') -> None:
        if self.thread is None:
            raise ValueError("Thread is not set!")

        self.client.beta.threads.messages.create(
            thread_id=self.thread.id,
            role=role,
            content=content,
            file_ids=file_obj_ids
        )

    def run_assistant(self) -> Run:

        if self.assistant is None:
            raise ValueError(
                "Assistant is not set. Cannot run assistant without an assistant.")

        if self.thread is None:
            raise ValueError(
                "Thread is not set!")

        self.run: Run = self.client.beta.threads.runs.create(
            thread_id=self.thread.id,
            assistant_id=self.assistant.id,
        )
        return self.run

    # Polling 
    def wait_for_completion(self, run: Run) -> list[ThreadMessage]:

            if run is None:
                raise ValueError("Run is not set!")

            # while run.status in ["in_progress", "queued"]:
            while run.status not in ["completed", "failed"]:
                run_status = client.beta.threads.runs.retrieve(
                    thread_id=self.thread.id,
                    run_id=run.id
                )
                time.sleep(3)  # Wait for 3 seconds before checking again
                print(f"Status: {run_status.status}")

                if run_status.status == 'completed':
                    print("Run completed.")
                    return client.beta.threads.messages.list(thread_id=thread.id)
                    break
                elif run_status.status == 'requires_action' and run_status.required_action is not None:
                    print(f"Function Calling ...")
                    self.call_required_functions(
                        toolCalls=run_status.required_action.submit_tool_outputs.model_dump(),
                        thread_id=thread.id,
                        run_id=run.id
                        )
                elif run.status == "failed":
                    print("Run failed.")
                    break
                else:
                    print(f"Waiting for the Assistant to process...: {run.status}")

    # Function to call the required functions
    def call_required_functions(self, toolCalls, thread_id: str, run_id: str):
            
            tool_outputs = []

            # for toolcall in toolCalls:
            for toolcall in toolCalls["tool_calls"]:
                function_name = toolcall['function']['name']
                function_args = json.loads(toolcall['function']['arguments'])
                
                if function_name in available_functions:

                    # Displaying the message with values
                    print(f"calling function {function_name} with args:")
                    for key, value in function_args.items():
                        print(f"{key}: {value}")

                    if function_name in available_functions:
                        function_to_call = available_functions[function_name]
                        print("function_to_call >>>>>" ,function_to_call)
                        output = function_to_call(**function_args)

                        print("Output Status", output)
                        
                        tool_outputs.append({
                            "tool_call_id": toolcall['id'],
                            "output": output,
                        })

                else:
                    raise ValueError(f"Unknown function: {function_name}")               
    
            print('submit_tool_outputs >>>>>' ,tool_outputs,) 
            # Submit tool outputs and update the run
            client.beta.threads.runs.submit_tool_outputs(
                thread_id=thread_id,
                run_id=run_id,
                tool_outputs=tool_outputs
                )

### Function To Get User Prompt and Perform Assistant Operations

In [299]:
ai_travel_maanger: TravelAIChat = TravelAIChat(assistant=travel_agent, thread=thread)

def call_travel_assistant(prompt: str ,file_ids: list[str] = []) -> list[ThreadMessage]:

    # Add a message to the thread
    ai_travel_maanger.add_message_to_thread(
        role="user",
        content=prompt,
        file_obj_ids=file_ids
    )

    # Run the assistant
    run: Run = ai_travel_maanger.run_assistant()

    # Wait for the assistant to complete
    messages: list[ThreadMessage] = ai_travel_maanger.wait_for_completion(run)

    return messages


### Case 1: Annotate Map Using Parallel Function Calling

In [300]:
chat_res = call_travel_assistant(prompt="Share 2 travel destinations in UAR next to each other and show them on map?")

Status: in_progress
Waiting for the Assistant to process...: queued
Status: in_progress
Waiting for the Assistant to process...: queued
Status: requires_action
Function Calling ...
calling function add_markers with args:
longitudes: [54.3705, 55.2708]
latitudes: [24.4824, 25.2048]
labels: ['Sheikh Zayed Grand Mosque, Abu Dhabi', 'Burj Khalifa, Dubai']
function_to_call >>>>> <function add_markers at 0x11bdd6160>
Output Status Markers added successfully.
calling function update_map with args:
longitude: 54.8227
latitude: 24.8436
zoom: 8
function_to_call >>>>> <function update_map at 0x11bdd60c0>
Output Status Map Updated.
submit_tool_outputs >>>>> [{'tool_call_id': 'call_WC7L3u9YFh9znFVq8GNwobfR', 'output': 'Markers added successfully.'}, {'tool_call_id': 'call_XcYD0ph3y6zxB9b2Xiu9Pzdm', 'output': 'Map Updated.'}]
Status: in_progress
Waiting for the Assistant to process...: queued
Status: in_progress
Waiting for the Assistant to process...: queued
Status: in_progress
Waiting for the Assi

In [301]:
# Show Results

print('\n \n')

for m in (chat_res.data):
    (print(m.role,":", ), print(m.content[0].text.value))
    print()

create_map(map_state=map_state, markers_state=markers_state)


 

assistant :
I have two travel destinations for you in the United Arab Emirates that are close to each other:

1. **Sheikh Zayed Grand Mosque** in Abu Dhabi: A remarkable architectural work that is also one of the largest mosques in the world, famous for its intricate Islamic art and carvings as well as majestic structure.

2. **Burj Khalifa** in Dubai: The tallest building in the world, offering breathtaking views of Dubai from its observation decks and a testament to modern engineering and design.

Both have been marked on the map for you:

- Sheikh Zayed Grand Mosque, Abu Dhabi is marked as (A).
- Burj Khalifa, Dubai is marked as (B).

The map has been updated to center between these two landmarks for your convenience.

user :
Share 2 travel destinations in UAR next to each other and show them on map?



### Case 2: Retrival & Contepratorde I

In [302]:
files_manager: TravelFiles = TravelFiles()

# Upload File and get the id

upload_file_id: str = files_manager.upload_file(file_path="sample_airbnb_receipt.pdf")

In [303]:
print(upload_file_id)

file-sOaSHFgBTMt00B0YxYHNzhNR


In [304]:
file_chat_res = call_travel_assistant(file_ids=[upload_file_id], prompt="We were three friends who stayed at this airbnb. What's my share?")

Status: in_progress
Waiting for the Assistant to process...: queued
Status: in_progress
Waiting for the Assistant to process...: queued
Status: in_progress
Waiting for the Assistant to process...: queued
Status: completed
Run completed.


In [305]:
# Here it uses retrival to get info and code inteprator to make calculations

print('\n \n')

for m in (file_chat_res.data):
    (print(m.role,":", ), print(m.content[0].text.value))
    print()


 

assistant :
Based on the Airbnb receipt, the total cost for the stay was $600.00. If you were three friends sharing this cost equally, here's the calculation for each person's share:

Total cost: $600.00
Number of friends: 3
Share per person: $600.00 / 3 = $200.00

Your share of the Airbnb cost would be $200.00.

user :
We were three friends who stayed at this airbnb. What's my share?

assistant :
I have two travel destinations for you in the United Arab Emirates that are close to each other:

1. **Sheikh Zayed Grand Mosque** in Abu Dhabi: A remarkable architectural work that is also one of the largest mosques in the world, famous for its intricate Islamic art and carvings as well as majestic structure.

2. **Burj Khalifa** in Dubai: The tallest building in the world, offering breathtaking views of Dubai from its observation decks and a testament to modern engineering and design.

Both have been marked on the map for you:

- Sheikh Zayed Grand Mosque, Abu Dhabi is marked as (A).
- 

### Key Notes:

1. The Travel Assistant Smartly decided when to use parallel function calling and When code inteprator and retrival
2. In case 1 it used function calling and annotated the map along with travel suggestions
3. In case 2 it only uses retrival and code inteprator to caluuculate our share
